#### Import Libraries and Load Credentials

In [6]:
import pandas as pd
import mysql.connector
from dotenv import load_dotenv
import os
from tqdm.notebook import tqdm

# Load .env file
load_dotenv()

DB_HOST = os.getenv('DB_HOST')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME = os.getenv('DB_NAME')

print(" Credentials loaded securely from .env")

 Credentials loaded securely from .env


#### Load and Inspect the CSV File



In [11]:
csv_path = 'ethiopian_msme_loans_realistic.csv'
df = pd.read_csv(csv_path)

print(f" Loaded {len(df):,} records from CSV")
print("Sample:")
display(df.head(3))

# Connect to database
try:
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME
    )
    cursor = conn.cursor()

    insert_query = """
        INSERT IGNORE INTO msme_loans (
            loan_id, borrower_name, region, sector, loan_date,
            loan_amount_etb, interest_rate, term_months,
            mobile_transactions, income_variability, credit_score, `default`
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    print(" Inserting data into MySQL...")
    inserted = 0
    skipped = 0

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Inserting"):
        try:
            cursor.execute(insert_query, tuple(row))
            inserted += 1
        except mysql.connector.Error as err:
            if err.errno == 1062:  # Duplicate primary key
                skipped += 1
            else:
                print(f"Warning: {err}")

    conn.commit()
    print(f"\n Load complete!")
    print(f"   Inserted: {inserted:,} records")
    print(f"   Skipped (duplicates): {skipped}")

except Exception as e:
    print(f" Error: {e}")
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()
    print(" Connection closed")

 Loaded 1,000 records from CSV
Sample:


,loan_id,borrower_name,region,sector,loan_date,loan_amount_etb,interest_rate,term_months,mobile_transactions,income_variability,credit_score,default
0,ee892190-394b-4d35-9f37-7ff855327432,Habtamu Samuel,Sidama,Trade/Retail (Market Vendors/Suq),2025-12-15,383074.22,18.26,6,394,0.176,718,0
1,108e04ac-99c6-407f-ac81-4510c42eafc8,Zewditu Worku,Dire Dawa,Trade/Retail (Market Vendors/Suq),2025-06-18,390097.59,23.02,36,248,0.261,776,0
2,b7aa53b6-e900-466f-a083-bf8857dda51e,Worknesh Tewodros,Afar,Trade/Retail (Market Vendors/Suq),2025-10-02,123260.26,16.86,36,153,0.345,654,0


 Inserting data into MySQL...


Inserting:   0%|          | 0/1000 [00:00<?, ?it/s]


 Load complete!
   Inserted: 1,000 records
   Skipped (duplicates): 0
 Connection closed


#### Verify the Data in MySQL

In [12]:
try:
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME
    )

    # Total count
    total_records = pd.read_sql("SELECT COUNT(*) AS count FROM msme_loans", conn).iloc[0]['count']
    print(f"Total records now in msme_loans table: {total_records:,}")

    # Sample data
    print("\nSample 10 records:")
    sample = pd.read_sql("SELECT * FROM msme_loans LIMIT 10", conn)
    display(sample)

    # Quick portfolio summary
    print("\nPortfolio Summary:")
    summary = pd.read_sql("""
        SELECT 
            ROUND(AVG(loan_amount_etb), 2) AS avg_loan_etb,
            ROUND(AVG(interest_rate), 2) AS avg_rate,
            COUNT(*) AS total_loans,
            SUM(`default`) AS defaults,
            ROUND(100.0 * SUM(`default`) / COUNT(*), 2) AS default_rate_pct
        FROM msme_loans
    """, conn)
    display(summary)

except Exception as e:
    print(f"Verification error: {e}")
finally:
    if conn.is_connected():
        conn.close()

C:\Users\Eldu\AppData\Local\Temp\ipykernel_2740\3935753178.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_records = pd.read_sql("SELECT COUNT(*) AS count FROM msme_loans", conn).iloc[0]['count']


Total records now in msme_loans table: 1,000

Sample 10 records:


C:\Users\Eldu\AppData\Local\Temp\ipykernel_2740\3935753178.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sample = pd.read_sql("SELECT * FROM msme_loans LIMIT 10", conn)


,loan_id,borrower_name,region,sector,loan_date,loan_amount_etb,interest_rate,term_months,mobile_transactions,income_variability,credit_score,default
0,00059b9d-a87d-41ff-b1fa-7e6afa236dbb,Eskinder Bereket,Somali,Construction,2024-10-24,41401.5,20.99,18,192,0.222,527,0
1,00ebf725-2f1e-48dc-b4f9-02b74580a4ae,Sisay Tsegaye,Afar,Trade/Retail (Market Vendors/Suq),2025-08-06,333668.0,26.28,12,211,0.250,672,0
2,0151ccb0-1b6f-4a9e-9a98-b7154c60b543,Girma Eyob,Afar,Digital/Tech (Mobile Money Agents),2024-05-30,335860.0,23.68,36,151,0.340,585,0
3,01862029-ab8a-4015-a727-3710203d54c9,Addis Amanuel,Amhara,Manufacturing (Textiles/Food Processing),2024-09-06,137002.0,19.60,36,213,0.170,751,0
4,01a6c260-efc0-4273-bd39-7fe7b819b15d,Jemal Birhanu,Central Ethiopia,Manufacturing (Textiles/Food Processing),2024-05-02,212272.0,24.47,36,494,0.201,668,0
5,01b4cdfc-31f4-46ea-8c8a-bd6989b4db80,Kaleb Ermias,Tigray,Livestock,2025-04-23,340537.0,25.79,18,182,0.211,670,0
6,01f268f5-6553-45aa-bdbc-34f8c8ddd9bb,Solomon Tariku,South West Ethiopia,Trade/Retail (Market Vendors/Suq),2026-01-03,239356.0,24.10,18,157,0.286,517,0
7,02223c10-f82f-4dc3-9c35-730b49d52453,Bezawit Dawit,Afar,Livestock,2024-06-28,354439.0,20.98,12,131,0.317,616,0
8,0267c983-f763-4b4d-8ada-c12203748864,Solomon Solomon,South West Ethiopia,Services (Transport/Tourism),2025-07-25,292637.0,19.13,6,83,0.230,592,0
9,02929d3d-96af-428a-978c-b5a03b668afa,Leulit Elias,Amhara,Construction,2024-01-10,34199.9,19.62,24,225,0.149,738,0



Portfolio Summary:


C:\Users\Eldu\AppData\Local\Temp\ipykernel_2740\3935753178.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  summary = pd.read_sql("""


,avg_loan_etb,avg_rate,total_loans,defaults,default_rate_pct
0,206753.75,20.14,1000,146.0,14.6
